In [148]:
import pandas as pd
import numpy as np
import difflib

In [149]:
cd C:\Users\pgsva\PycharmProjects\DTU\MBML24

C:\Users\pgsva\PycharmProjects\DTU\MBML24


In [150]:
data_national = pd.read_csv('data/raw/COMEPELDA_aggregate_v1.00.csv')
data_european = pd.read_csv('data/raw/1999-2019_CHES_dataset_means.csv')
country_codes = pd.read_csv('data/raw/countries.csv')  
party_names = pd.read_csv('data/raw/partyNames_ncv2.csv')  

In [151]:
# Add country codes to national data
data_national = pd.merge(data_national, country_codes, left_on='cName', right_on='Country', how='left')

In [153]:
# add country codes to european data
data_european = pd.merge(data_european, country_codes, left_on='country', right_on='ID', how='left')

In [157]:
data_european = pd.merge(data_european, party_names, left_on='party_id', right_on='partyID', how='left')

In [159]:
test_data = pd.merge(data_european, data_national, left_on=["Country", 'partyName'], right_on=["cName", 'pName'], how='outer', indicator=True)

In [160]:
T1 = test_data[test_data['_merge'] == "left_only"]
T2 = test_data[test_data['_merge'] == "right_only"]
T3 = test_data[test_data['_merge'] == "both"]

In [161]:
T1.describe()

,country,eastwest,eumember,year,expert,party_id,cmp_id,vote,seat,electionyear,...,pdFlagNofCand,cSeats,pcSeats,pdSeats,IndepCand01,cAddLisbon,pAddLisbon,pAlliance01,IDauxElec,ID_y
count,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,825.000000,1000.000000,994.000000,1055.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,14.689100,0.633175,0.925118,2009.665403,11.737441,1477.287204,50328.951515,11.007010,11.646278,2008.293839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,9.702013,0.482167,0.263325,6.854747,4.696710,969.017320,29178.184741,10.793449,14.015692,6.904809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,0.000000,0.000000,1999.000000,2.000000,103.000000,11110.000000,0.010000,0.000000,1996.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,6.000000,0.000000,1.000000,2002.000000,8.000000,612.000000,31110.000000,3.367500,1.500000,2002.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,12.000000,1.000000,1.000000,2010.000000,11.000000,1202.000000,41221.000000,6.900000,6.650000,2009.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,24.000000,1.000000,1.000000,2014.000000,14.000000,2405.000000,82220.000000,15.325000,16.375000,2014.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,40.000000,1.000000,1.000000,2019.000000,27.000000,4009.000000,97952.000000,49.280000,64.599998,2019.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
T2.describe()

,country,eastwest,eumember,year,expert,party_id,cmp_id,vote,seat,electionyear,...,pdFlagNofCand,cSeats,pcSeats,pdSeats,IndepCand01,cAddLisbon,pAddLisbon,pAlliance01,IDauxElec,ID_y
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000971,55.541748,7.288350,1.607767,0.017476,0.267961,0.028155,0.125243,473.464078,12.633981
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.031159,29.284199,10.924841,2.586790,0.131099,0.602911,0.165497,0.331155,270.470781,9.010684
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,219.000000,6.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,70.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,410.000000,8.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,78.000000,9.000000,2.000000,0.000000,0.000000,0.000000,0.000000,824.000000,20.000000
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,99.000000,43.000000,27.000000,1.000000,4.000000,1.000000,1.000000,849.000000,40.000000


In [163]:
T3.describe()

,country,eastwest,eumember,year,expert,party_id,cmp_id,vote,seat,electionyear,...,pdFlagNofCand,cSeats,pcSeats,pdSeats,IndepCand01,cAddLisbon,pAddLisbon,pAlliance01,IDauxElec,ID_y
count,3548.000000,3548.000000,3548.000000,3548.000000,3548.00000,3548.000000,3188.000000,3479.000000,3479.000000,3548.000000,...,3548.000000,3548.000000,3548.000000,3548.000000,3548.0,3548.000000,3548.000000,3548.000000,3548.000000,3548.000000
mean,14.257046,0.667418,0.932074,2008.860767,11.67221,1431.417418,56256.428168,14.722521,16.477379,2007.356821,...,0.002255,49.996054,7.317926,1.904453,0.0,0.219278,0.040304,0.089064,469.625141,14.257046
std,8.929596,0.471205,0.251654,6.483942,4.46601,891.910603,27883.808808,13.488499,17.725466,6.615806,...,0.047438,26.160873,7.268335,3.305113,0.0,0.500747,0.205119,0.284877,270.465378,8.929596
min,1.000000,0.000000,0.000000,1999.000000,2.00000,102.000000,11520.000000,0.100000,0.000000,1996.000000,...,0.000000,5.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,4.000000,1.000000
25%,8.000000,0.000000,1.000000,2002.000000,8.00000,803.000000,32530.000000,3.900000,2.000000,2002.000000,...,0.000000,22.000000,2.000000,0.000000,0.0,0.000000,0.000000,0.000000,219.000000,8.000000
50%,11.000000,1.000000,1.000000,2010.000000,11.00000,1107.000000,51901.000000,8.900000,8.800000,2007.000000,...,0.000000,51.000000,4.000000,1.000000,0.0,0.000000,0.000000,0.000000,374.000000,11.000000
75%,26.000000,1.000000,1.000000,2014.000000,15.00000,2601.000000,92210.000000,26.850000,30.700001,2011.000000,...,0.000000,73.000000,12.000000,2.000000,0.0,0.000000,0.000000,0.000000,822.000000,26.000000
max,40.000000,1.000000,1.000000,2019.000000,27.00000,4005.000000,97951.000000,55.040000,63.599998,2019.000000,...,1.000000,99.000000,33.000000,33.000000,0.0,4.000000,2.000000,1.000000,849.000000,40.000000


In [164]:
3071

3071

In [179]:
#data_national["alt_partyName"] = data_national["pName"].astype(str).apply(lambda x: difflib.get_close_matches(x, data_european[data_national["cName"] == data_european["Country"]]["partyName"].astype(str)))

data_national["alt_partyName"] = [difflib.get_close_matches(data_national.loc[idx, "pName"], data_european[data_national.loc[idx, "cName"] == data_european["Country"]]["partyName"].astype(str)) for idx in range(len(data_national))]


In [206]:
data_european[(data_european["Country"] == "Italy") & (data_european["partyName"] == "Partito dei Pensionati")]

,country,eastwest,eumember,year,expert,party_id,cmp_id,party,vote,seat,...,chesversion,ID,Abbreviation,Country,CountryAbrev,partyID,partyAbrev,partyName,partyNameEng,alt_partyName
546,8,1,1,2006,6,830,NaN,PP,0.9,0.0,...,2020.1,8,IT,Italy,IT,830.0,PP,Partito dei Pensionati,Pensioners’ Party,"[Partito Pensionati, Partito Pensionati, Parti..."


In [180]:
data_national[["cName", "pName", "alt_partyName"]]

,cName,pName,alt_partyName
0,Austria,Freiheitliche Partei Österreichs,"[Freiheitliche Partei Österreichs, Freiheitlic..."
1,Austria,Liberales Forum,"[Liberales Forum, Liberales Forum, Liberales F..."
2,Austria,Sozialdemokratische Partei Österreichs,"[Sozialdemokratische Partei Österreichs, Sozia..."
3,Austria,Österreichische Volkspartei,"[Österreichische Volkspartei, Österreichische ..."
4,Austria,Die Grünen – Die Grüne Alternative,[]
...,...,...,...
1801,Sweden,Socialdemokraterna,"[Sverigedemokraterna, Sverigedemokraterna, Sve..."
1802,Sweden,Miljöpartiet de Gröna,"[Miljöepartiet de Gröna, Miljöepartiet de Grön..."
1803,Sweden,Centerpartiet,"[Centerpartiet, Centerpartiet, Centerpartiet]"
1804,Sweden,Feministiskt initiativ,[Feministiskt initiativ]


In [166]:
data_european["alt_partyName"] = data_european["partyName"].astype(str).apply(lambda x: difflib.get_close_matches(x, data_national["pName"].astype(str)))

In [167]:
data_european[["Country", "partyName", "alt_partyName"]]

,Country,partyName,alt_partyName
0,Belgium,Front National,"[Front national, Front national, Front national]"
1,Belgium,Christelijke Volkspartij,"[Christelijke Volkspartij, Christelijke Volksp..."
2,Belgium,Christen-Democratisch en Vlaams,"[Christen Democratisch Appèl, Christen Democra..."
3,Belgium,Open Vlaamse Liberalen en Democraten,[Partij voor Vrijheid en Vooruitgang | Vlaamse...
4,Belgium,Parti Réformateur Libéral,"[Parti Réformateur Libéral, Partidul Democrat-..."
...,...,...,...
1573,Cyprus,Kinima Sosialdimokraton EDEK,"[Kinima Sosialdimokraton EDEK, Kinima Sosialdi..."
1574,Cyprus,Kinima Oikologon Perivallontiston,[]
1575,Cyprus,Dimokratikó Kómma,"[Dimokratikó Kómma, Dimokratikó Kómma, Dimokra..."
1576,Cyprus,Ethniko Laiko Metopo,[]
